## Reinforcement Learning

Beim Reinforcement Learning gibt es im Gegensatz zum klassischen Machine Learning kein vorgegebenes Datenset

Die Daten werden während des Prozesses erzeugt und direkt verarbeitet

Hier werden keinen Zusammenhänge gesucht, sondern der Agent bekommt Punkte für Aktionen die er gut macht, und verliert optional für andere Aktionen

Begriffe:
- Umgebung
    - Der Prozess, das Programm
    - Hier ist festgelegt, welche Aktionen möglich sind und Punkte geben

- Prozess
    - Das Programm/der Ablauf
    - Der Ablauf läuft die ganze Zeit und gibt uns die Daten
 
- Agent
    - "Spieler" der die Punkte bekommt
    - Determiniert, was gute und schlechte Aktionen sind
 
Packages:
- Gym
- Stable Baselines 3

In [1]:
import gymnasium as gym

## Vordefinierte Umgebungen

Es gibt ein paar vorgegebene Umgebungen, die wir zum testen verwenden können

Hier geht es darum, die Pole von dem Cart möglichst gerade zu halten (90° Winkel). Dies erfolgt durch links- und rechts Bewegungen von dem Cart

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [24]:
env = gym.make("CartPole-v1", render_mode="human")

## Inhalte der Umgebung

Funktionen:
- init: Aufsetzen der Umgebung
- step(): Einen Schritt machen
- reset(): Umgebung zurücksetzen
- render(): Zeichnet die Umgebung (in einer GUI)
- close(): Beendet die Umgebung

Begriffe:
- Action Space: Aktionen, die der Agent durchführen darf (Controls des Spiels)
- Observation Space: Die Datenpunkte die beobachtet werden (Game State)

## Der Agent

In [11]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [25]:
def testEnv(env, durchgaenge):
    for d in range(durchgaenge):
        state = env.reset() # Am Anfang des Durchgangs die Umgebung zurücksetzen
        score = 0
        terminate = False

        # Diese Schleife spielt das Spiel
        while not terminate:
            env.render()  # Zeichne die GUI
            action = env.action_space.sample()  # Nimm eine Random Action
            state, reward, terminate, done, info = env.step(action)  # Führe einen Step mit der Random Action aus
            # Hier kommen 5 Rückgabewerte heraus: Den Game State (Observation Space), die Belohnung, Verloren Ja/Nein, Spiel zu Ende (optional), Info: Beliebige Daten
            score += reward
        print(f"Durchgang {d + 1}: Score {score}")
    env.close()

In [26]:
testEnv(env, 10)

Durchgang 1: Score 26.0
Durchgang 2: Score 16.0
Durchgang 3: Score 13.0
Durchgang 4: Score 14.0
Durchgang 5: Score 54.0
Durchgang 6: Score 12.0
Durchgang 7: Score 16.0
Durchgang 8: Score 27.0
Durchgang 9: Score 31.0
Durchgang 10: Score 14.0


## RL Modell

Trainingsalgorithmus
- PPO (Proximal Policy Optimization)
- https://stable-baselines.readthedocs.io/en/master/guide/algos.html

DummyVecEnv (Dummy Vector Environment)
- Simulation mehrerer Durchgänge gleichzeitig
- Zeit sparen

Policy
- Bestimmt wie der Algorithmus
- Basiert auf TensorFlow (hat ein TF Model dahinter)
- https://stable-baselines.readthedocs.io/en/master/modules/policies.html

In [27]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [32]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device


In [33]:
model.learn(total_timesteps=200000)

-----------------------------
| time/              |      |
|    fps             | 2025 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1371        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008044812 |
|    clip_fraction        | 0.0834      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00146    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.86        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 53.7        |
-----------------------------------------
----------------------------------

In [34]:
model.save("Models/CartPoleModel")

## Model testen

In [18]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO

In [19]:
env = gym.make("CartPole-v1", render_mode="human")

In [20]:
model = PPO.load("Models/CartPoleModel.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [40]:
def testEnvPredict(env, durchgaenge):
    for d in range(durchgaenge):
        state = env.reset() # Am Anfang des Durchgangs die Umgebung zurücksetzen
        score = 0
        terminate = False
        done = False

        # Diese Schleife spielt das Spiel
        while not (terminate or done):
            env.render()  # Zeichne die GUI
            if len(state) == 2:
                action = model.predict(np.array(state[0]))
            else:
                action = model.predict(np.array(state))
            state, reward, terminate, done, info = env.step(action[0])  # Führe einen Step mit der Random Action aus
            # Hier kommen 5 Rückgabewerte heraus: Den Game State (Observation Space), die Belohnung, Verloren Ja/Nein, Spiel zu Ende (optional), Info: Beliebige Daten
            score += reward
        print(f"Durchgang {d + 1}: Score {score}")
    env.close()

In [41]:
testEnvPredict(env, 5)

Durchgang 1: Score 500.0
Durchgang 2: Score 500.0
Durchgang 3: Score 500.0
Durchgang 4: Score 500.0
Durchgang 5: Score 500.0
